# Importing Libraries

In [ ]:
import os
import re
from sklearn.model_selection import StratifiedKFold
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import shutil
from functools import partial
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from kaggle_datasets import KaggleDatasets

# Reading Old Data

In [ ]:
# Old Data 
# Making a df of all he images
os.listdir('../input/cassavaold/train/train/cbb')

df = pd.DataFrame(columns = ['image_id', 'label'])

df = pd.concat([pd.DataFrame({'image_id': '../input/cassavaold/train/train/cbb/'+pd.Series(os.listdir('../input/cassavaold/train/train/cbb')),
                             'label':0}),
                pd.DataFrame({'image_id': '../input/cassavaold/train/train/cbsd/'+pd.Series(os.listdir('../input/cassavaold/train/train/cbsd')),
                             'label':1}),
               pd.DataFrame({'image_id': '../input/cassavaold/train/train/cgm/'+pd.Series(os.listdir('../input/cassavaold/train/train/cgm')),
                             'label':2}),
               pd.DataFrame({'image_id': '../input/cassavaold/train/train/cmd/'+pd.Series(os.listdir('../input/cassavaold/train/train/cmd')),
                             'label':3}),
               pd.DataFrame({'image_id': '../input/cassavaold/train/train/healthy/'+pd.Series(os.listdir('../input/cassavaold/train/train/healthy')),
                             'label':4})]
                           , axis=0, ignore_index=True)
display(df.head())
print(df.shape)


# Reading New Cassava Data

In [ ]:
df2 = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df2['image_id'] = "../input/cassava-leaf-disease-classification/train_images/"+df2['image_id']
df2.shape

# Merging Both

In [ ]:
df_merged = pd.concat([df,df2], axis=0, ignore_index=True)
df_merged.shape

In [ ]:
plt.figure(figsize=(10,10), dpi=80)
for img, label in zip(df_merged['image_id'], df_merged['label']): 
        display(label)
        display(tf.keras.preprocessing.image.load_img(img))
        break

# Code For Writing TFRecord

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(feature0, feature1, feature2):
  feature = {
      'image': _bytes_feature(feature0),
      'image_name': _bytes_feature(feature1),
      'label': _int64_feature(feature2)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
PATH = f'{database_base_path}train_images/'
IMGS = df_merged['image_id'].tolist()
N_FILES = 15
HEIGHT, WIDTH = (512, 512)
IMG_QUALITY = 100
print('No of Images - ', len(IMGS))
train = df_merged.copy()
display(train.head())


folds = StratifiedKFold(n_splits=N_FILES, shuffle=True, random_state=123)
train['file'] = -1

for fold_n, (train_idx, val_idx) in enumerate(folds.split(train, train['label'])):
    print('File: %s has %s samples' % (fold_n+1, len(val_idx)))
    train['file'].loc[val_idx] = fold_n
    
train.to_csv('train.csv', index=False)

# Writing to TFRecords
for tfrec_num in range(N_FILES):
    print('\nWriting TFRecord %i of %i...'%(tfrec_num, N_FILES))
    samples = train[train['file'] == tfrec_num]
    n_samples = len(samples)
    print(f'{n_samples} samples')
    
    with tf.io.TFRecordWriter('Id_train%.2i-%i.tfrec'%(tfrec_num, n_samples)) as writer:
        for row in samples.itertuples():
            label = row.label
            image_name = row.image_id
            img_path = image_name
            
            img = cv2.imread(img_path)
            img = cv2.resize(img, (HEIGHT, WIDTH))
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, IMG_QUALITY))[1].tostring()
            
            example = serialize_example(img, str.encode(image_name), label)
            writer.write(example)